#### Метод Ритца в задаче Штурма-Лиувилля.

$(\frac{1}{2 + \frac{x}{3}}u')' + (\exp(\frac{x}{5}))u = \lambda u$

$u'(-1) - 0.6 u(-1) = u'(1) + 0.65u(1) = 0$

На обоих концах промежутка 3 тип задачи.

In [109]:
import numpy as np
from scipy.special import jacobi
from scipy.integrate import quadrature
import copy

import pandas as pd

a, b = -1, 1
n = 7
a1, a2, b1, b2 = -0.6, -1, 0.65, 1


def p(x):
    return 1 / (2 + x / 3)


def q(x):
    return np.exp(x / 5)


def scalar(y, z):
    return quadrature(lambda x: y(x) * z(x), a, b)[0]


def d(f):
    eps = 1e-10
    return lambda x: (f(x + eps) - f(x - eps)) / (2 * eps)


def d2(f):
    eps = 1e-5
    return lambda x: (f(x + eps) - f(x - eps)) / (2 * eps)


def integrate(y, z):
    def fun(x):
        return p(x) * d(y)(x) * d(z)(x) + q(x) * y(x) * z(x)

    tmp1 = a1 / a2 * p(a) * y(a) * z(a)
    tmp2 = b1 / b2 * p(b) * y(b) * z(b)

    return quadrature(fun, a, b)[0] + tmp1 + tmp2

Метод Ритца

In [110]:
def w(x, k):
    tmp = np.sqrt((2 * k - 1) / 2)
    return tmp * jacobi(n=k - 1, alpha=0, beta=0)(x)


def w_k(k):
    return lambda x: w(x, k)

G = np.zeros((n, n))
G_L = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        G[i, j] = scalar(w_k(i + 1), w_k(j + 1))
        G_L[i, j] = integrate(w_k(i + 1), w_k(j + 1))

lambdas = np.linalg.eig(G_L)[0]
lambdas.sort()

print("Матрица G_l и собственные значения, полученные методом Ритца ", lambdas)
table = pd.DataFrame(G_L)
table

/usr/local/lib/python3.6/dist-packages/scipy/integrate/quadrature.py:251: AccuracyWarning: maxiter (50) exceeded. Latest difference = 3.820935e-08
  AccuracyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/integrate/quadrature.py:251: AccuracyWarning: maxiter (50) exceeded. Latest difference = 7.626377e-08
  AccuracyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/integrate/quadrature.py:251: AccuracyWarning: maxiter (50) exceeded. Latest difference = 8.129043e-07
  AccuracyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/integrate/quadrature.py:251: AccuracyWarning: maxiter (50) exceeded. Latest difference = 1.591099e-06
  AccuracyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/integrate/quadrature.py:251: AccuracyWarning: maxiter (50) exceeded. Latest difference = 7.384242e-06
  AccuracyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/integrate/quadrature.py:251: AccuracyWarning: maxiter (50) exceeded. Latest difference = 7.757886e-06
  AccuracyWarning)
/usr

Матрица G_l и собственные значения, полученные методом Ритца  [  1.26481057   2.81114489   6.56442286  13.1193276   23.21779468
  96.33802148 180.17773695]


,0,1,2,3,4,5,6
0,1.325966,0.045413,0.719924,-0.107518,0.957862,-0.135034,1.151201
1,0.045413,3.484011,-0.382052,3.824738,-0.656721,4.788504,-0.789546
2,0.719924,-0.382052,10.234469,-1.650361,12.495725,-2.208693,15.013075
3,-0.107518,3.824738,-1.650361,24.671556,-4.285512,29.881981,-5.294012
4,0.957862,-0.656721,12.495725,-4.285512,49.879847,-8.803250,59.084298
5,-0.135034,4.788504,-2.208693,29.881981,-8.803250,88.944916,-15.718252
6,1.151201,-0.789546,15.013075,-5.294012,59.084298,-15.718252,144.952493


Оценка собственных значений

In [111]:
x = np.linspace(-1, 1, 1000)
p_max = max(map(p, x))
p_min = min(map(p, x))
q_max = max(map(q, x))
q_min = min(map(q, x))

nu1 = 0.760936
nu2 = 1.9300737

print("Нижняя оценка на lambda1: ", nu1**2 * p_min + q_min)
print("Верхняя оценка на lambda1: ", nu1**2 * p_max + q_max)
print("Нижняя оценка на lambda2: ", nu2**2 * p_min + q_min)
print("Верхняя оценка на lambda2: ", nu2**2 * p_max + q_max)

def y(nu):
  C = (b2 * nu * np.sin(nu) - b1 * np.cos(nu)) / (b1 * np.sin(nu) + b2 * nu * np.cos(nu))
  f = lambda x: np.cos(nu*x) + C * np.sin(nu*x)
  return lambda x: f(x) / np.sqrt(scalar(f, f))


norm_eig_f_1 = y(nu1)
norm_eig_f_2 = y(nu2)

print("Приближенное начение первого собственного числа: ", integrate(norm_eig_f_1, norm_eig_f_1))

print("Приближенное значение второго собственного числа", integrate(norm_eig_f_2, norm_eig_f_2))

Нижняя оценка на lambda1:  1.0668837228334103
Верхняя оценка на lambda1:  1.5688169158177698
Нижняя оценка на lambda2:  2.415238390548706
Верхняя оценка на lambda2:  3.456513450619184
Приближенное начение первого собственного числа:  1.2739823470264757
Приближенное значение второго собственного числа 2.80639980229313


In [112]:
A = np.dot(np.linalg.inv(G_L), G)

w, v = np.linalg.eig(A)  # собственные числа и векторы матрицы H
l_max = 0
for x in w:
    if abs(x) > abs(l_max):
        l_max = x
print("Минимальное собственное число встроенным методом", 1 / l_max)

Минимальное собственное число встроенным методом 1.2648105699765704


In [113]:

v = [1, 0, 0, 0, 0, 0, 0]
q = [v]
lambdas = [v]

i = 1
eps = 10 ** (-3)
lam_1, lam_2 = 1, 0

while np.abs(lam_1 - lam_2) > eps and i!=15:
    q.append(np.dot(A, q[i - 1]))
    lam_2 = copy.deepcopy(lam_1)
    lam_1 = np.dot(q[i], q[i-1])/np.dot(q[i-1], q[i-1])
    i += 1

w_max = lam_1
print("Минимальное собственное число методом обратных итераций", 1 / w_max)
print("Абсолютная погрешность", abs(w_max - l_max))
print("Количество итераций:", i)


Минимальное собственное число методом обратных итераций 1.2648843942764105
Абсолютная погрешность 4.614482634879291e-05
Количество итераций: 4
